# Emotion Detection 

In [45]:
# Importing the necessary libraries 
import cv2
import matplotlib.pyplot as plt
from skimage import io
import os
import numpy

In [46]:
# Reading the files in the filepath and displaying the filenames in the path.
filepath=r"..\images"
files=os.listdir(filepath)
print(files)
print(len(files))

['subject02_happy.png', 'subject02_surprised.png', 'subject03_happy.png', 'subject03_surprised.png', 'subject04_happy.png', 'subject04_surprised.png', 'subject05_happy.png', 'subject05_surprised.png', 'subject06_happy.png', 'subject06_surprised.png', 'subject07_happy.png', 'subject07_surprised.png', 'subject08_happy.png', 'subject08_surprised.png', 'subject09_happy.png', 'subject09_surprised.png', 'subject10_happy.png', 'subject10_surprised.png', 'subject11_happy.png', 'subject11_surprised.png', 'subject12_happy.png', 'subject12_surprised.png', 'subject13_happy.png', 'subject13_surprised.png', 'subject14_happy.png', 'subject14_surprised.png', 'subject15_happy.png', 'subject15_surprised.png']
28


In [47]:
# The model uses haarcascade xml files to process the image(faces) data.
def detect_face(img):
    fd=cv2.CascadeClassifier(r"..\haarcascade_frontalface_alt.xml")
    faces=fd.detectMultiScale(img,1.3,5)
    #print("..............")
    #print(faces)
    if len(faces)==0:
        return None
    (x,y,w,h)=faces[0]
    return img[y:y+h,x:x+w], (x,y,w,h)

In [48]:
def load_data(filepath):
    xdata=[]
    ydata=[]
    files=os.listdir(filepath)
    for file in files:
        if 'happy' or 'surprised' in file:
            ipath=filepath+'\\'+file
            img=io.imread(ipath)
            face,area=detect_face(img)
            #print("**************************")
            #print(face)
            #print('---------------------------')
            #print(area)
            if face is not None:
                face=face.astype(numpy.float32)
                face=face/(face.max())
                face=cv2.resize(face,(120,120))
                face=numpy.ndarray.flatten(face)
                xdata.append(face)
                if 'happy' in file:
                    ydata.append('happy')
                elif 'surprised' in file:
                    ydata.append('surprised')
    return xdata,ydata

In [49]:
#The xdata i.e. the train data and the ydata i.e. the test data are returned.
xdata,ydata=load_data(filepath)

In [50]:
print(len(xdata))
print(len(ydata))

28
28


In [51]:
# The classifier is trained using scikit-learn
from sklearn import neighbors
alg=neighbors.KNeighborsClassifier(n_neighbors=3)
xdata=numpy.array(xdata)
ydata=numpy.array(ydata)
alg.fit(xdata,ydata)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [52]:
#The algorithm accuracy is calculated here.
alg.score(xdata,ydata)

0.8928571428571429

In [53]:
# The model is pickled.
from sklearn.externals import joblib
joblib.dump(alg,'emotion.pkl')

['emotion.pkl']

In [54]:
#loading 'emotion.pkl' pickle file.
model=joblib.load('emotion.pkl')

In [55]:
# prediction function.
def predict_face(img):
    fd=cv2.CascadeClassifier(r"..\haarcascade_frontalface_alt.xml")
    shape=img.shape
    if len(shape)==3:
        img=cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    faces=fd.detectMultiScale(img,1.3,5)
    if len(faces)==0:
        return None
    (x,y,w,h)=faces[0]
    return img[y:y+h,x:x+w], (x,y,w,h)

In [64]:
def prediction(img):
    face,area=predict_face(img)
    face=face.astype(numpy.float32)
    print(face)
    face=face/(face.max())
    face=cv2.resize(face,(120,120))
    print(face.shape)
    face=numpy.ndarray.flatten(face)
    face = face.reshape(1,-1)
    out=model.predict(face)
    (x,y,w,h)=area
    cv2.rectangle(img,(x,y),(x+w,y+h),[255,0,0],2)
    cv2.putText(img,out[0],(x,y),cv2.FONT_HERSHEY_PLAIN,3.5,[0,0,255],2)
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return out[0]

In [65]:
# The emotion prediction done on a single image.
img=cv2.imread(r"..\surprise1.jpg")
img1=img.astype(numpy.float32)
#print(img1.shape)
prediction(img)

[[216. 216. 215. ... 207. 207. 207.]
 [215. 215. 216. ... 207. 207. 207.]
 [215. 215. 216. ... 207. 207. 207.]
 ...
 [ 99.  99.  99. ...  96. 112. 141.]
 [100. 100. 100. ...  89.  99. 118.]
 [101. 101. 101. ...  89.  89. 100.]]
(120, 120)


'surprised'